In [16]:
# Import necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
import os, csv
import sys
from time import time, strftime

In [17]:
# Connect to SQlite database
def connect_db(database_name_with_path):
    cursor = None
    try:
      sqliteConnection = sqlite3.connect(database_name_with_path)
      cursor = sqliteConnection.cursor()
      print(f'SQlite connected with {db}')
    except:
      sys.stderr.write("Failed to connect to database")
    return cursor

SQlite connected with historicaldata.db


In [18]:
# Test connection query
try:
  query = "SELECT COUNT(*) FROM all_historical;"
  cursor.execute(query)
  print(cursor.fetchall())

except:
  sys.stderr.write("Failed to execute query")

[(296751,)]


In [19]:
# labelling threshold percentile value - if in a day, this much price raise occurs on day's starting price, then label buy 
gain_threshold = 20
row_data_threshold = 25
#csv filename
csv_filename = "min_day_rows-" + (str)(row_data_threshold) + "-min_price_gain-" + (str)(gain_threshold) + "pc-" + (str)(time()) + ".csv"
uncompressed_csv_filename = "uc-" + csv_filename

# setup database connection
cursor = connect_db('historicaldata.db')
if (cursor == None):
    sys.stderr.write("Failed to get cursor...")
    exit(1)


In [20]:
# let's create the dataset
all_historical_dataset = list()
tickers_list = list()
ticker_dates_list = list()
ticker_date_data_list = list()

dataset = list()
uncompressed_dataset = list()
#try:
query = "select distinct(ticker) as ticker from all_historical where 1 order by ticker;"
cursor.execute(query)
tickers_list = [list(i) for i in cursor.fetchall()]
ticker_data = list()
for ticker in tickers_list:
    
    ticker_query = "select distinct(strftime('%Y-%m-%d', time/1000, 'unixepoch')) as ticker_date from all_historical where ticker='" + ticker[0] + "' order by ticker_date;"
    cursor.execute(ticker_query)
    ticker_dates_list = [list(j) for j in cursor.fetchall()]

    date_data = list()
    for ticker_date in ticker_dates_list:
        
        ticker_date_query = "select strftime('%H:%M', time/1000, 'unixepoch') as ticker_time, volume, volume_weighted_average, time from all_historical where ticker='" + ticker[0] + "' and strftime('%Y-%m-%d', time/1000, 'unixepoch') = '" + ticker_date[0] + "' order by ticker_time;" 
        cursor.execute(ticker_date_query)
        ticker_date_data_list = [list(k) for k in cursor.fetchall()]

        first_hm_price = 0.0 # actually it's 0
        previous_hm = (np.nan, np.nan, np.nan, np.nan) # actually it's 0
        row_data = list()
        row_label = list()
        label = "no_buy"
        f = 0
        is_first_hm_price_set = False
        for ticker_date_data_row in ticker_date_data_list:
            # Let's skip the 0 hours data
            f = f + 1
            if ((ticker_date_data_row[0]) == '00:00'):
                continue
            
            # if (first_hm_price == 0. and first_hm_price < (float)(ticker_date_data_list[f][2])):
            if (not(is_first_hm_price_set)):
                first_hm_price = (float)(ticker_date_data_list[f][2])
                is_first_hm_price_set = True
            
            # If previous time is more than 5 seconds but not 0000 hours, we loop to impute values by previous rows
            if (previous_hm != (np.nan, np.nan, np.nan, np.nan)):
                time_diff = (int)((ticker_date_data_row[3] - previous_hm[3])/(1000 * 60 * 5))
                if time_diff > 1:
                    for td in range (time_diff-1):
                        previous_hm_0_plus_5 = ((int)(previous_hm[0])+5)
                        if (previous_hm_0_plus_5%100==60):
                            previous_hm_0_plus_5 = (int)(previous_hm[0])+45 #(+100-55)
                        row_data.append(((previous_hm_0_plus_5), (float)(previous_hm[1]), (float)(previous_hm[2])))
                        row_label.append((label))  ### Since, this is just copying previous rows, retaining price raise label
                        previous_hm = ((previous_hm_0_plus_5), (float)(previous_hm[1]), (float)(previous_hm[2]), (previous_hm[3]))
                        # time_diff = time_diff - 1
                        uncompressed_dataset.append((ticker[0], ticker_date[0], (previous_hm_0_plus_5), (float)(previous_hm[1]), (float)(previous_hm[2]), label))

                # parallely, we label the row as buy if there's a 30% raise in price, default=no_buy
                
                price_change_percentage = float((float(ticker_date_data_row[2] - first_hm_price) * 100)/float(first_hm_price))
                if price_change_percentage >= gain_threshold:  # price gain/raise
                    label = "buy"

            ticker_date_data_row_0 = ticker_date_data_row[0].replace(":","")
            row_data.append(((int)(ticker_date_data_row_0), (float)(ticker_date_data_row[1]), (float)(ticker_date_data_row[2])))
            row_label.append((label))
            previous_hm = ((int)(ticker_date_data_row_0), (float)(ticker_date_data_row[1]), (float)(ticker_date_data_row[2]),(ticker_date_data_row[3]))
            uncompressed_dataset.append((ticker[0], ticker_date[0], (int)(ticker_date_data_row_0), (float)(ticker_date_data_row[1]), (float)(ticker_date_data_row[2]), label))
        #print(row_data)
        #exit(1)
        # avoid adding data that has zero records or less than 60% from a day (25.2/42 records, starting at 4:30 ending at 8:00)
        if (len(row_data) > row_data_threshold):
            date_data.append((ticker[0], ticker_date[0], row_data, row_label))

            with open(csv_filename, 'a', newline='') as file:
                csvwriter = csv.writer(file)
                csvwriter.writerow((ticker[0], ticker_date[0], row_data, row_label))
            file.close()
    ticker_data.append((date_data))
#print(ticker_data)
dataset = ticker_data
#print(dataset)

print("Dataset prepared successfully and loaded in variable: dataset. A csv file (" + csv_filename + ") is also created in same directory.")

with open(uncompressed_csv_filename, 'a', newline='') as file:
    csvwriter = csv.writer(file)
    for uncompressed_datarow in uncompressed_dataset:
        csvwriter.writerow(uncompressed_datarow)
file.close()

print("An uncompressed version of the dataset is stored in csv file " + uncompressed_csv_filename + " in the same directory.")



Dataset prepared successfully and loaded in variable: dataset. A csv file (min_day_rows-25-min_price_gain-20pc-1721711727.7495308.csv) is also created in same directory.
An uncompressed version of the dataset is stored in csv file uc-min_day_rows-25-min_price_gain-20pc-1721711727.7495308.csv in the same directory.


In [14]:
cursor.close()

In [ ]:
### SQL QUERIES ###
'''
select ticker, strftime('%Y-%m-%d', "time"/1000, 'unixepoch') as ticker_date, strftime('%H:%M', "time"/1000, 'unixepoch') as ticker_time, volume_weighted_average from CDAK where 1 order by ticker, ticker_date, ticker_time;

select ticker , strftime('%Y-%m-%d', "time"/1000, 'unixepoch') as ticker_date from all_historical where 1 group by ticker order by ticker, ticker_date;

select ticker , max(strftime('%Y-%m-%d', "time"/1000, 'unixepoch')) as max_ticker_date, min(strftime('%Y-%m-%d', "time"/1000, 'unixepoch')) as min_ticker_date from all_historical where 1 group by ticker order by ticker;

select ticker , max(strftime('%H:%M', "time"/1000, 'unixepoch')) as max_ticker_time, min(strftime('%H:%M', "time"/1000, 'unixepoch')) as min_ticker_time, avg(strftime('%H:%M', "time"/1000, 'unixepoch')) as avg_ticker_time from all_historical where 1 group by ticker order by ticker;

select ticker , strftime('%H:%M', "time"/1000, 'unixepoch') as ticker_time, count(strftime('%H:%M', "time"/1000, 'unixepoch')) as count_ticker_time from all_historical where 1 group by ticker, ticker_time order by ticker, ticker_time;


select distinct(ticker) as ticker from all_historical where 1 order by ticker;

select distinct(strftime('%Y-%m-%d', "time"/1000, 'unixepoch')) as ticker_date from all_historical where ticker="AAU" order by ticker_date;

select time, strftime('%Y-%m-%d', "time"/1000, 'unixepoch') as ticker_date, strftime('%H:%M', time/1000, 'unixepoch') as ticker_time, volume, volume_weighted_average from all_historical where ticker="AAU" order by ticker_time, ticker_date;

select ticker, strftime('%Y-%m-%d %H:%M:%S', time/1000, 'unixepoch') as ticker_date from all_historical where time>=1671494400000 and time<=1671557700000 ;

select ticker, strftime('%Y-%m-%d %H:%M:%S', time/1000, 'unixepoch') as ticker_date, strftime('%H:%M', time/1000, 'unixepoch') as ticker_time, volume, volume_weighted_average from all_historical where strftime('%Y-%m-%d', time/1000, 'unixepoch') between '2023-11-12' and '2023-11-25' order by ticker, ticker_date;


'''
